# 6. Interação

_"Um gráfico não é 'desenhado' de uma vez por todas; mas sim 'construído' e reconstruído até que revele todas as relações constituídas pela interação dos dados. As melhores operações gráficas são aquelas performadas pelo próprio tomador de decisões."_ — [Jacques Bertin](https://books.google.com/books?id=csqX_xnm4tcC)

A visualização fornece formas poderosas de analisar os dados. Uma única imagem, no entanto, oferece respostas para, na melhor das hipóteses, um punhado de perguntas. Através da *interação* podemos transformar imagens estáticas em ferramentas para exploração: destacando pontos de interesse, ampliando para revelar padrões refinados e vinculando várias visualizações para explorar relacionamentos multidimensionais.

No cerne da interação está a noção de *seleção*: um meio de indicar ao computador quais elementos ou regiões nos interessam. Por exemplo, podemos passar o mouse sobre um ponto, clicar em múltiplas marcas ou desenhar um retângulo delimitante ao redor de uma região para destacar subconjuntos de dados para análise mais aprofundada.

Junto às codificações visuais e transformações de dados, o Altair oferece uma abstração de *seleção* para interações de autoria. Essas seleções abrangem três aspectos:

1. Manipulação de eventos de entrada para selecionar pontos ou regiões de interesse, como movimentação do mouse, cliques, arrastos, rolagens e toques.

2. Generalizar a partir da entrada para formular uma regra de seleção (ou [*predicado*](https://en.wikipedia.org/wiki/Predicate_%28mathematical_logic%29)) que determine se um registro está ou não dentro de uma seleção.

3. Usar o predicado de seleção para configurar dinamicamente uma visualização ao aplicar _codificações condicionais_, _transformações de filtro_ ou _domínios de escala_.

Esse notebook introduz seleções interativas e explora como utilizá-las para realizar uma variedade de técnicas de interação, tais quais consultas dinâmicas, panorâmica e zoom, detalhes sob demanda, pincelamento e vinculação.

*Esse notebook é parte do [currículo de visualização de dados](https://github.com/uwdata/visualization-curriculum).*


In [2]:
import pandas as pd
import altair as alt

# 6.1. Conjuntos de Dados (Pedro Miguel)

Vamos visualizar uma variedade de conjuntos de dados da coleção [vega-datasets](https://github.com/vega/vega-datasets):
- Um conjunto de dados sobre carros (`cars`) da década de 1970 e início da década de 1980;
- Um conjunto de dados sobre filmes (`movies`), previamente utilizado no notebook de [Transformação de Dados](https://github.com/uwdata/visualization-curriculum/blob/master/altair_data_transformation.ipynb);
- Um conjunto de dados contendo dez anos de preços de ações do [S&P 500](https://en.wikipedia.org/wiki/S%26P_500) (`sp500`);
- Um conjunto de dados sobre ações (`stocks`) de companhias de tecnologia;
- Um conjunto de dados sobre voos (`flights`), incluindo horário de partida, distância e atraso de chegada.

In [3]:
cars = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/cars.json'
movies = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/movies.json'
sp500 = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/sp500.csv'
stocks = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/stocks.csv'
flights = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/flights-5k.json'

# 6.2. Introduzindo Seleções

Vamos começar com uma seleção básica: simplesmente clicando em um ponto para destacá-lo. Utilizando o conjunto de dados `cars`, vamos começar com um gráfico de dispersão de cavalos de potência versus milhas por galão, com uma codificação de cor para o número de cilindros no motor do carro.

Ademais, criaremos uma instância de seleção chamando `alt.selection_single()`, indicando que queremos uma seleção definida por um *único valor*. Por padrão, a seleção utiliza um clique de mouse para determinar o valor selecionado. Para registrar a seleção com um gráfico, devemos adicionar isso usando o método `.add_selection()`.

Uma  vez que nossa seleção foi definida, podemos utilizá-la como um parâmetro para *codificações condicionais*, que aplicam diferentes codificações dependendo se um registro de dados está dentro ou fora da seleção. Por exemplo, considere o seguinte trecho de código:

In [ ]:
color=alt.condition(selection, 'Cylinders:O', alt.value('grey'))

Essa definição de codificação indica que pontos contidos em `selection` deveriam ser coloridos de acordo com o campo `Cylinder`, enquanto pontos não selecionados deveriam ser coloridos por uma cor padrão `grey` (cinza). Uma seleção vazia inclui *todos* os pontos, e então inicialmente todos os pontos serão coloridos.

*Tente clicar em diferentes pontos no gráfico abaixo. O que acontece? (Clique no fundo para limpar o estado de seleção e retornar a uma seleção "vazia")*

In [5]:
selection = alt.selection_single();

alt.Chart(cars).mark_circle().add_selection(
    selection
).encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color=alt.condition(selection, 'Cylinders:O', alt.value('grey')),
    opacity=alt.condition(selection, alt.value(0.8), alt.value(0.1))
)

<ipython-input-5-6db95e64aa5e>:1: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use selection_point instead.
  selection = alt.selection_single();
<ipython-input-5-6db95e64aa5e>:3: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  alt.Chart(cars).mark_circle().add_selection(


alt.Chart(...)

É claro, destacar pontos individuais um a um não é particularmente animador! Como veremos, no entanto, seleções de valores individuais fornecem um útil bloco de construção para interações mais poderosas. Ademais, seleções de valores individuais são apenas um dos três tipos fornecidos pelo Altair:

- `selection_single` - seleciona um único valor discreto, por padrão em eventos de clique.
- `selection_multi` - seleciona múltiplos valores discretos. O primeiro valor é selecionado por clique de mouse e valores adicionais são alternados usando shift-click.
- `selection_interval` - seleciona uma gama contínua de valores, inicialmente pelo arrastar do mouse.

Vamos comparar cada um dos tipos de seleções lado a lado. Para manter nosso código compacto vamos primeiro definir uma função (`plot`) que gera uma especificação para gráficos de dispersão assim como o exemplo acima. Podemos passar uma seleção para a função `plot` para aplicá-la ao gráfico:

In [6]:
def plot(selection):
    return alt.Chart(cars).mark_circle().add_selection(
        selection
    ).encode(
        x='Horsepower:Q',
        y='Miles_per_Gallon:Q',
        color=alt.condition(selection, 'Cylinders:O', alt.value('grey')),
        opacity=alt.condition(selection, alt.value(0.8), alt.value(0.1))
    ).properties(
        width=240,
        height=180
    )

Vamos usar a nossa função `plot` para criar três gráficos variantes, um por tipo de seleção.

O primeiro gráfico (`single`) replica nosso exemplo anterior. O segundo gráfico (`multi`) suporta interações shift-click para alternar inclusão de múltiplos pontos dentro da seleção. O terceiro gráfico (`interval`) gera uma região de seleção (ou *pincel*) ao arrastar o mouse. Uma vez criado, você pode arrastar o pincel para selecionar diferentes pontos ou rolar quando o cursor estiver dentro do mesmo para dimensionar (ampliar) o tamanho do pincel.

*Tente interagir com cada um dos gráficos abaixo!*

In [7]:
alt.hconcat(
  plot(alt.selection_single()).properties(title='Single (Click)'),
  plot(alt.selection_multi()).properties(title='Multi (Shift-Click)'),
  plot(alt.selection_interval()).properties(title='Interval (Drag)')
)

<ipython-input-7-060afde6be3b>:2: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use selection_point instead.
  plot(alt.selection_single()).properties(title='Single (Click)'),
<ipython-input-6-ceafea467174>:2: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  return alt.Chart(cars).mark_circle().add_selection(
<ipython-input-7-060afde6be3b>:3: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use selection_point instead.
  plot(alt.selection_multi()).properties(title='Multi (Shift-Click)'),


alt.HConcatChart(...)

Os exemplos acima usam interações padrão (click, shift-click, arrasto) para cada tipo de seleção. Podemos aprofundar na customização de interações fornecendo especificações do evento de input usando a [sintaxe do seletor de evento do Vega](https://vega.github.io/vega/docs/event-streams/). Por exemplo, podemos modificar nossos gráficos `single` e `multi` para ativar em eventos de `mouseover` em vez de eventos de `click`.

*Pressione a tecla shift no segundo gráfico para "pintar" com dados!*

In [8]:
alt.hconcat(
  plot(alt.selection_single(on='mouseover')).properties(title='Single (Mouseover)'),
  plot(alt.selection_multi(on='mouseover')).properties(title='Multi (Shift-Mouseover)')
)

<ipython-input-8-517206693e99>:2: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use selection_point instead.
  plot(alt.selection_single(on='mouseover')).properties(title='Single (Mouseover)'),
<ipython-input-6-ceafea467174>:2: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  return alt.Chart(cars).mark_circle().add_selection(
<ipython-input-8-517206693e99>:3: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use selection_point instead.
  plot(alt.selection_multi(on='mouseover')).properties(title='Multi (Shift-Mouseover)')


alt.HConcatChart(...)

Agora que cobrimos as seleções básicas do Altair, vamos fazer um tour pelas várias técnicas de interação que elas permitem!

# 6.3. Consultas Dinâmicas

*Consultas Dinâmicas* permitem explorações rápidas e reversíveis de dados para isolar padrões de interesse. Como definido por [Ahlberg, Williamson & Shneiderman](https://www.cs.umd.edu/~ben/papers/Ahlberg1992Dynamic.pdf), uma consulta dinâmica:

- representa graficamente uma consulta;
- fornece limites visíveis no alcance da consulta;
- fornece uma representação gráfica dos dados e do resultado da consulta;
- dá um feedback imediato do resultado após ajuste da consulta;
- permite novatos a iniciar os trabalhos com pouco treinamento.

Uma abordagem comum é manipular os parâmetros da consulta utilizando widgets de interface de usuário padrão como sliders, botões de opção e menus suspensos. Para gerar widgets de consulta dinâmicos, podemos aplicar uma operação de vinculação de seleção a um ou mais campos de dados que desejamos consultar.

Vamos construir um gráfico de dispersão interativo que utiliza uma consulta dinâmica para filtrar a exibição. Dado um gráfico de dispersão de avaliações de filmes (do Rotten Tomatoes e IMDb) podemos adicionar uma seleção sobre o campo `Major_Genre` que habilite filtrar filmes por gênero.

Para começar, vamos extrair os gêneros únicos (não nulos) dos dados dos filmes:

In [10]:
df = pd.read_json(movies) # load movies data
genres = df['Major_Genre'].unique() # get unique field values
genres = list(filter(lambda d: d is not None, genres)) # filter out None values
genres.sort() # sort alphabetically

Para uso posterior, também vamos definir uma lista de valores exclusivos `MPAA_Rating`:

In [11]:
mpaa = ['G', 'PG', 'PG-13', 'R', 'NC-17', 'Not Rated']

Agora criamos uma seleção `single` vinculada a um menu suspenso.

*Use o menu de consulta dinâmica abaixo para explorar os dados. Como as classificações variam por gênero? Como você revisaria o código para filtrar `MPAA_Rating` (G, PG, PG-13, etc.) ao invés de `Major_Genre`?*

In [ ]:
selectGenre = alt.selection_single(
    name='Select', # nome da seleção 'Select'
    fields=['Major_Genre'], # limitando a seleção a Major_Genre field
    init={'Major_Genre': genres[0]}, # utilizando a primeira entrada do gênero como valor inicial
    bind=alt.binding_select(options=genres) # ligar a um menu de valores únicos de gênero
)

alt.Chart(movies).mark_circle().add_selection(
    selectGenre
).encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q',
    tooltip='Title:N',
    opacity=alt.condition(selectGenre, alt.value(0.75), alt.value(0.05))
)


<ipython-input-16-9d58f2bf452e>:1: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use selection_point instead.
  selectGenre = alt.selection_single(


TypeError: altair.vegalite.v5.schema.core.SelectionParameter() got multiple values for keyword argument 'value'

Nossa construção acima faz uso de múltiplos aspectos de seleções:

- Demos um nome à seleção (`Select`). Esse nome não é exigido, mas permite que influenciemos os rótulos de texto do menu de consulta dinâmigo gerado. *(O que acontece se você remover o nome? Teste!)*
- Nós restringimos a seleção a um único campo de dados (`Major_Genre`). Anteriomente usamos uma seleção `single`, a seleção era mapeada para pontos de dados individuais. Ao limitar a seleção a um campo específico, podemos selecionar _todos_ os pontos de dados cujo valor do campo `Major_Genre` corresponde ao único valor selecionado.
- Iniciamos `init=...` a seleção para um valor inicial.
- Nós vinculamos (`bind`) a seleção a um widget de interface, neste caso um menu suspenso via `binding_select`.
- Como antes, usamos uma codificação condicional para controlar a opacidade do canal.

# 6.3.1. Vinculando Seções a Entradas Múltiplas

Uma instância de seleção pode ser vinculada a *múltiplos* widgets de consultas dinâmicas. Vamos modificar o exemplo acima para fornecer filtros para *ambos* `Major_Genre` e `MPAA_Rating`, usando botões de opção ao invés de um menu. Nossa seleção `single` agora está definida sobre um único *par* de gêneros e valores de classificação MPAA.

*Procure por conjunções surpreendentes de gênero e classificações indicativas. Há algum filme de terror classificado como G ou PG?*

In [ ]:
# Seleção de valor único sobre pares [Major_Genre, MPAA_Rating]
# Utilizar valores específicos fixados como valores iniciais selecionados
selection = alt.selection_single(
    name='Select',
    fields=['Major_Genre', 'MPAA_Rating'],
    init={'Major_Genre': 'Drama', 'MPAA_Rating': 'R'},
    bind={'Major_Genre': alt.binding_select(options=genres), 'MPAA_Rating': alt.binding_radio(options=mpaa)}
)

# scatter plot, modificar a opacidade com base na seleção
alt.Chart(movies).mark_circle().add_selection(
    selection
).encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q',
    tooltip='Title:N',
    opacity=alt.condition(selection, alt.value(0.75), alt.value(0.05))
)

<ipython-input-17-a42312080268>:3: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use selection_point instead.
  selection = alt.selection_single(


TypeError: altair.vegalite.v5.schema.core.SelectionParameter() got multiple values for keyword argument 'value'

*Fatos curiosos: A classificação PG-13 não existia quando filmes como [Tubarão](https://www.imdb.com/title/tt0073195/) e [Tubarão 2](https://www.imdb.com/title/tt0077766/) foram lançados. O primeiro filme a receber uma classificação PG-13 foi [Amanhecer Violento](https://www.imdb.com/title/tt0087985/), de 1984.*